In [27]:
import pandas as pd

In [28]:
# read in the zori data
zori = pd.read_csv('heat_zori_metro_clean.csv', index_col=0)

# only cols I need
zori = zori[['name', 'pop_2023', 'date', 'lat', 'lng', 'zori']]

# change 'date' from the last day of the month to the first day of the month
zori['date'] = pd.to_datetime(zori['date'])
zori['date'] = zori['date'] + pd.offsets.MonthBegin(-1)

# create a yoy % change column for zori in each metro
zori['zori_yoy'] = zori.groupby('name')['zori'].pct_change(12)

zori

,name,pop_2023,date,lat,lng,zori,zori_yoy
0,United States,335893238,2018-01-01,NaN,NaN,1405.842990,NaN
1,"New York, NY",19498249,2018-01-01,40.6943,-73.9249,2520.028880,NaN
2,"Los Angeles, CA",12799100,2018-01-01,34.1141,-118.4068,2190.975308,NaN
3,"Chicago, IL",9262825,2018-01-01,41.8375,-87.6866,1509.843282,NaN
4,"Dallas, TX",8100037,2018-01-01,32.7935,-96.7667,1289.240890,NaN
...,...,...,...,...,...,...,...
4450,"Hartford, CT",1151543,2024-09-01,41.7661,-72.6834,1920.139081,0.074701
4451,"Tucson, AZ",1063162,2024-09-01,32.1541,-110.8787,1621.090639,0.021585
4452,"Rochester, NY",1052087,2024-09-01,43.1680,-77.6162,1456.431005,0.040440
4453,"Urban Honolulu, HI",989408,2024-09-01,NaN,NaN,2693.099648,0.046516


In [29]:
# read in demand data
demand = pd.read_csv('demand_metro_wide.csv')
                     
# only msa
demand = demand[demand['RegionType'] == 'msa']

# drop cols I don't need
demand.drop(
    columns=['RegionID', 'SizeRank', 'RegionType', 'StateName'],
    inplace=True
)

# rename name
demand = demand.rename(columns={'RegionName': 'name'})

#  melt the data
demand = demand.melt(
    id_vars='name',
    var_name='date', 
    value_name='demand'
    )

# change 'date' from the last day of the month to the first day of the month
demand['date'] = pd.to_datetime(demand['date'])
demand['date'] = demand['date'] + pd.offsets.MonthBegin(-1)

demand


,name,date,demand
0,"New York, NY",2020-06-01,53.0
1,"Los Angeles, CA",2020-06-01,134.0
2,"Chicago, IL",2020-06-01,97.0
3,"Dallas, TX",2020-06-01,108.0
4,"Houston, TX",2020-06-01,46.0
...,...,...,...
44727,"Snyder, TX",2024-10-01,8.0
44728,"Atchison, KS",2024-10-01,NaN
44729,"Fairfield, IA",2024-10-01,NaN
44730,"Vermillion, SD",2024-10-01,54.0


In [30]:
# merge the data
demand_rent = pd.merge(
    zori,
    demand,
    on=['name', 'date'],
    how='inner'
    )

demand_rent

,name,pop_2023,date,lat,lng,zori,zori_yoy,demand
0,"New York, NY",19498249,2020-06-01,40.6943,-73.9249,2646.400473,-0.010733,53.0
1,"Los Angeles, CA",12799100,2020-06-01,34.1141,-118.4068,2343.983177,0.003254,134.0
2,"Chicago, IL",9262825,2020-06-01,41.8375,-87.6866,1616.518818,0.006820,97.0
3,"Dallas, TX",8100037,2020-06-01,32.7935,-96.7667,1392.014212,0.011164,108.0
4,"Houston, TX",7510253,2020-06-01,29.7860,-95.3885,1386.519255,0.000992,46.0
...,...,...,...,...,...,...,...,...
2803,"Hartford, CT",1151543,2024-09-01,41.7661,-72.6834,1920.139081,0.074701,145.0
2804,"Tucson, AZ",1063162,2024-09-01,32.1541,-110.8787,1621.090639,0.021585,32.0
2805,"Rochester, NY",1052087,2024-09-01,43.1680,-77.6162,1456.431005,0.040440,149.0
2806,"Urban Honolulu, HI",989408,2024-09-01,NaN,NaN,2693.099648,0.046516,81.0


In [31]:
# save the data
demand_rent.to_csv('demand_rent_metro.csv', index=False)